In [20]:
!pip install pulp

from itertools import combinations
import pulp

def maximize_coupons(item_prices, coupons, coupons_once=False):
    """
    Splits items into batches to maximize the total benefit from coupons with different thresholds and benefits.

    Parameters:
    - item_prices: List of item prices (list of floats or ints).
    - coupons: List of coupon dictionaries, each with 'threshold' and 'benefit'.
    - coupons_once: Boolean indicating if coupons can be used only once (True) or multiple times (False).

    Returns:
    - batches: List of batches, where each batch is a dictionary with keys:
        - 'items': List of item prices in the batch.
        - 'total': Total price of the batch.
        - 'coupon_threshold': The threshold of the coupon applied to the batch (if any).
        - 'coupon_benefit': The benefit of the coupon applied to the batch (if any).
    """
    N = len(item_prices)
    item_prices = [float(price) for price in item_prices]
    M = len(coupons)
    item_indices = range(N)
    coupon_indices = range(M)

    # Generate all possible subsets of items that could potentially use a coupon
    subsets = []
    for r in range(1, N+1):
        for items in combinations(item_indices, r):
            total_price = sum(item_prices[i] for i in items)
            applicable_coupons = []
            for c_idx, coupon in enumerate(coupons):
                if total_price >= coupon['threshold']:
                    applicable_coupons.append(c_idx)
            if applicable_coupons:
                subsets.append({
                    'items': items,
                    'total_price': total_price,
                    'applicable_coupons': applicable_coupons
                })
            else:
                # Subsets that don't meet any coupon thresholds
                subsets.append({
                    'items': items,
                    'total_price': total_price,
                    'applicable_coupons': [None]
                })

    # Create the ILP model
    prob = pulp.LpProblem("Maximize_Coupons", pulp.LpMaximize)

    # Decision variables: whether to select a subset with a specific coupon
    # Keyed by (subset index, coupon index)
    x = {}
    for s_idx, subset in enumerate(subsets):
        for c_idx in subset['applicable_coupons']:
            var_name = f"x_{s_idx}_{c_idx}"
            x[(s_idx, c_idx)] = pulp.LpVariable(var_name, cat='Binary')

    # Objective function: Maximize total coupon benefit
    prob += pulp.lpSum(
        x[s_c] * (0 if s_c[1] is None else coupons[s_c[1]]['benefit'])
        for s_c in x
    )

    # Constraint: Each item must be included exactly once
    for i in item_indices:
        prob += pulp.lpSum(
            x[s_c]
            for s_c in x
            if i in subsets[s_c[0]]['items']
        ) == 1, f"Item_{i}_included_once"

    # Constraint: Each coupon can be used at most once if coupons_once is True
    if coupons_once:
        for c_idx in coupon_indices:
            prob += pulp.lpSum(
                x[s_c]
                for s_c in x
                if s_c[1] == c_idx
            ) <= 1, f"Coupon_{c_idx}_used_once"

    # Solve the ILP
    prob.solve()

    # Extract the selected batches
    batches = []
    for s_c in x:
        if pulp.value(x[s_c]) == 1:
            s_idx, c_idx = s_c
            subset = subsets[s_idx]
            items_in_batch = [item_prices[i] for i in subset['items']]
            coupon_applied = coupons[c_idx] if c_idx is not None else None
            batches.append({
                'items': items_in_batch,
                'total': subset['total_price'],
                'coupon_threshold': coupon_applied['threshold'] if coupon_applied else None,
                'coupon_benefit': coupon_applied['benefit'] if coupon_applied else 0
            })

    return batches

# Usage:

item_prices = [94.90, 7.21, 4.64, 30.5, 9.68, 29.35, 2.04, 3.50, 6.64, 6.57]
coupons = [
    {'threshold': 269, 'benefit': 30},
    # {'threshold': 169, 'benefit': 25},
    {'threshold': 89, 'benefit': 12},
    {'threshold': 39, 'benefit': 5},
    {'threshold': 19, 'benefit': 3},
]

# Set coupons_once=True to use coupons only once
batches_once = maximize_coupons(item_prices, coupons, coupons_once=True)

print("Batches when coupons can be used only once:")
total_benefit = 0
for i, batch in enumerate(batches_once, 1):
    if batch['coupon_threshold']:
        coupon_info = f"Coupon Applied: Threshold = {batch['coupon_threshold']}, Benefit = {batch['coupon_benefit']}"
    else:
        coupon_info = "No Coupon Applied"
    total_benefit += batch['coupon_benefit']
    print(f"Batch {i}: Items = {batch['items']}, Total = {batch['total']:.2f}, {coupon_info}")
print(f"Total Benefit: {total_benefit}")

# Set coupons_once=False to allow coupons to be used multiple times
batches_multiple = maximize_coupons(item_prices, coupons, coupons_once=False)

print("\nBatches when coupons can be used multiple times:")
total_benefit = 0
for i, batch in enumerate(batches_multiple, 1):
    if batch['coupon_threshold']:
        coupon_info = f"Coupon Applied: Threshold = {batch['coupon_threshold']}, Benefit = {batch['coupon_benefit']}"
    else:
        coupon_info = "No Coupon Applied"
    total_benefit += batch['coupon_benefit']
    print(f"Batch {i}: Items = {batch['items']}, Total = {batch['total']:.2f}, {coupon_info}")
print(f"Total Benefit: {total_benefit}")

Batches when coupons can be used only once:
Batch 1: Items = [94.9], Total = 94.90, Coupon Applied: Threshold = 89, Benefit = 12
Batch 2: Items = [30.5], Total = 30.50, Coupon Applied: Threshold = 19, Benefit = 3
Batch 3: Items = [9.68], Total = 9.68, No Coupon Applied
Batch 4: Items = [2.04], Total = 2.04, No Coupon Applied
Batch 5: Items = [6.64], Total = 6.64, No Coupon Applied
Batch 6: Items = [7.21, 4.64, 29.35, 3.5, 6.57], Total = 51.27, Coupon Applied: Threshold = 39, Benefit = 5
Total Benefit: 20

Batches when coupons can be used multiple times:
Batch 1: Items = [94.9], Total = 94.90, Coupon Applied: Threshold = 89, Benefit = 12
Batch 2: Items = [9.68, 29.35], Total = 39.03, Coupon Applied: Threshold = 39, Benefit = 5
Batch 3: Items = [30.5, 3.5, 6.57], Total = 40.57, Coupon Applied: Threshold = 39, Benefit = 5
Batch 4: Items = [7.21, 4.64, 2.04, 6.64], Total = 20.53, Coupon Applied: Threshold = 19, Benefit = 3
Total Benefit: 25
